In [1]:
import io
import requests
import torch
from PIL import Image
from rich import print
from pathlib import Path
import os
# Setup cache for Hugging Face models
os.environ['HF_HOME'] = '/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.cache'
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

In [2]:
#UTILS
test_png_dset_path = Path('/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test')
test_pathology_left_or_right = Path('/vol/biomedic3/bglocker/ugproj2324/nns20/datasets/VinDr-CXR/image_text_reasoning_datasets/test_pathology_left_or_right')
test_all_left_or_right = Path('/vol/biomedic3/bglocker/ugproj2324/nns20/datasets/VinDr-CXR/image_text_reasoning_datasets/test_all_left_or_right')


In [3]:
def setup_model() -> tuple:
    device = "cuda"
    dtype = torch.float16

    processor = AutoProcessor.from_pretrained("StanfordAIMI/CheXagent-8b", trust_remote_code=True)
    generation_config = GenerationConfig.from_pretrained("StanfordAIMI/CheXagent-8b")
    model = AutoModelForCausalLM.from_pretrained(
        "StanfordAIMI/CheXagent-8b", torch_dtype=dtype, trust_remote_code=True
    ).to(device)

    return processor, model, device, dtype, generation_config

def generate(images, prompt, processor, model, device, dtype, generation_config):
    inputs = processor(
        images=images, text=f" USER: <s>{prompt} ASSISTANT: <s>", return_tensors="pt"
    ).to(device=device, dtype=dtype)
    output = model.generate(**inputs, generation_config=generation_config)[0]
    response = processor.tokenizer.decode(output, skip_special_tokens=True)
    return response


In [4]:
model_params = setup_model() 


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [20]:
pathology = "Calcification"
# prompt = f"Give the location of the {pathology} in the image in a bounding box format."
# prompt = f"Localise the {pathology} in the image."
prompt = f"Is the {pathology} on the left or the right in the image?"
image_id = "cc657609a4dcf498ea39a9586d7ae9b3"

image_path = test_png_dset_path / f"{image_id}.png"
image = Image.open(image_path)
image = image.convert("RGB")

images = [image]
response = generate(images, prompt, *model_params)
print(response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Right

In [5]:
responses = []
with open(test_pathology_left_or_right) as f:
    f.readline() # skip header
    lines = f.readlines()
    for index, line in enumerate(lines):
        image_id,pathology = line.split(',')[:2]
        image = Image.open(test_png_dset_path / f"{image_id}.png").convert("RGB")
        # prompt = f"Given the image, localize the region of {pathology}."
        prompt = f"Localize the {pathology}, is it on the RIGHT or the LEFT or the RIGHT AND LEFT of the given image?"
        response = generate([image], prompt, *model_params)
        responses.append(f"{image_id},{pathology},{response}")

        if index % 10 == 0:
            with open('grl_RIGHT_or_LEFT_responses', 'w') as f:
                f.write('\n'.join(responses))
            print(f"Written up to index {index}")

/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.cache/modules/transformers_modules/StanfordAIMI/CheXagent-8b/4934e91451945c8218c267aae9c34929a7677829/processing_chexagent.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(pixel_values) for pixel_values in encoding_image_processor["pixel_values"]]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Written up to index 0

/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.cache/modules/transformers_modules/StanfordAIMI/CheXagent-8b/4934e91451945c8218c267aae9c34929a7677829/processing_chexagent.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(pixel_values) for pixel_values in encoding_image_processor["pixel_values"]]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-en

Written up to index 10

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Written up to index 2090

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Written up to index 2100

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
